In [22]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re

In [23]:
df = pd.read_csv('train.csv')

In [24]:
que_df = df.sample(30000)

In [25]:
df.dropna(inplace=True)

In [26]:
y = que_df['is_duplicate']

In [27]:
df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [28]:
que_df = que_df[['question1', 'question2']]

In [29]:
def preprocess(q):
  q = str(q).lower().strip()

  q = q.replace('%', ' percent')
  q = q.replace('$', ' dollar ')
  q = q.replace('₹', ' rupee ')
  q = q.replace('€', ' euro ')
  q = q.replace('@', ' at ')

  contractions = { 
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }

  q_decontracted = []

  for word in q.split():
      if word in contractions:
          word = contractions[word]

      q_decontracted.append(word)

  q = ' '.join(q_decontracted)
  q = q.replace("'ve", " have")
  q = q.replace("n't", " not")
  q = q.replace("'re", " are")
  q = q.replace("'ll", " will")
  
  # Removing HTML tags
  q = BeautifulSoup(q)
  q = q.get_text()
  
  # Remove punctuations
  pattern = re.compile('\W')
  q = re.sub(pattern, ' ', q).strip()

  
  return q


In [30]:
len(preprocess("<whar>do<you> want"))

7

In [31]:
que_df['question1'] = que_df['question1'].apply(preprocess)
que_df['question2'] = que_df['question2'].apply(preprocess)

In [32]:
que_df['len1'] = que_df['question1'].str.len()
que_df['len2'] = que_df['question2'].str.len()

In [33]:
que_df['word_cnt1'] = que_df['question1'].apply(lambda que : len(que.split(" "))) 
que_df['word_cnt2'] = que_df['question2'].apply(lambda que : len(que.split(" ")))
que_df.head()

,question1,question2,len1,len2,word_cnt1,word_cnt2
237544,what is screen resolution,what would be the screen resolution of a mirror,25,49,4,11
296630,what was the significance of the battle of som...,what was the significance of the battle of som...,121,119,23,23
372672,can a person with alzheimer disease participat...,can a person with alzheimer s disease particip...,65,69,10,12
187200,what is it like to live in an rv full time,what is it like to live in an rv and travel in it,42,49,11,13
218447,if one had 15 minutes to interview a successfu...,what are some good questions to ask an entrepr...,97,51,18,9


In [34]:
def common_words(row):
  w1 = set(map(lambda words : words.lower().strip(), row['question1'].split(" ")))
  w2 = set(map(lambda words : words.lower().strip(), row['question2'].split(" ")))
  return len(w1 & w2)

In [35]:
que_df['common'] = que_df.apply(common_words, axis=1)
que_df.head()

,question1,question2,len1,len2,word_cnt1,word_cnt2,common
237544,what is screen resolution,what would be the screen resolution of a mirror,25,49,4,11,3
296630,what was the significance of the battle of som...,what was the significance of the battle of som...,121,119,23,23,15
372672,can a person with alzheimer disease participat...,can a person with alzheimer s disease particip...,65,69,10,12,10
187200,what is it like to live in an rv full time,what is it like to live in an rv and travel in it,42,49,11,13,9
218447,if one had 15 minutes to interview a successfu...,what are some good questions to ask an entrepr...,97,51,18,9,8


In [36]:
def total_words(row):
  w1 = set(map(lambda words : words.strip(), row['question1'].split(" ")))
  w2 = set(map(lambda words : words.strip(), row['question2'].split(" ")))
  return (len(w1) + len(w2))

In [37]:
que_df['total_w'] = que_df.apply(total_words, axis=1)

In [38]:
que_df['word_share'] = round(que_df['common'] / que_df['total_w'], 2)

In [39]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tarun\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [40]:
!pip install distance

In [41]:
import distance
def fetch_features(row):
  q1 = row['question1']
  q2 = row['question2']

  SAFE_DIV = 0.00001
  STOP_WORDS = stopwords.words('english')
  token_features = [0.0] * 11

  q1_tokens = q1.split()
  q2_tokens = q2.split()

  if len(q1_tokens) == 0 or len(q2_tokens) == 0:
    return token_features

  q1_words = set([word for word in q1_tokens if word not in STOP_WORDS])
  q2_words = set([word for word in q2_tokens if word not in STOP_WORDS])

  q1_stop = set([word for word in q1_tokens if word in STOP_WORDS])
  q2_stop = set([word for word in q2_tokens if word in STOP_WORDS])

  common_word_cnt = len(q1_words.intersection(q2_words))
  common_stop_cnt = len(q1_stop.intersection(q2_stop))
  common_token_cnt = len(set(q1_tokens).intersection(set(q2_tokens)))

  token_features[0] = round(common_word_cnt / (min(len(q1_words), len(q2_words)) + SAFE_DIV), 2)
  token_features[1] = round(common_word_cnt / (max(len(q1_words), len(q2_words)) + SAFE_DIV), 2)

  token_features[2] = round(common_stop_cnt / (min(len(q1_stop), len(q2_stop)) + SAFE_DIV), 2)
  token_features[3] = round(common_stop_cnt / (max(len(q1_stop), len(q2_stop)) + SAFE_DIV), 2)

  token_features[4] = round(common_token_cnt / (min(len(q1_tokens), len(q2_tokens)) + SAFE_DIV), 2)
  token_features[5] = round(common_token_cnt / (max(len(q1_tokens), len(q2_tokens)) + SAFE_DIV), 2)

  token_features[6] = int(q1_tokens[-1] == q2_tokens[-1])
  token_features[7] = int(q1_tokens[0] == q2_tokens[0])

  token_features[8] = abs(len(q1_tokens) - len(q2_tokens))
  token_features[9] = round((len(q1_tokens) - len(q2_tokens)) / 2, 2)

  strs = list(distance.lcsubstrings(q1, q2))
  token_features[10] = round(len(strs[0]) / (min(len(q1), len(q2)) + 1), 2)
  return token_features

In [42]:
token_features = que_df.apply(fetch_features, axis=1)

In [43]:
que_df['cwc_min']       = list(map(lambda x: x[0], token_features))
que_df['cwc_max']       = list(map(lambda x: x[1], token_features))
que_df['csc_min']       = list(map(lambda x: x[2], token_features))
que_df['csc_max']       = list(map(lambda x: x[3], token_features))
que_df['ctc_min']       = list(map(lambda x: x[4], token_features))
que_df['ctc_max']       = list(map(lambda x: x[5], token_features))
que_df['last_word_eq']  = list(map(lambda x: x[6], token_features))
que_df['first_word_eq'] = list(map(lambda x: x[7], token_features))
que_df['abs_len_diff']  = list(map(lambda x: x[8], token_features))
que_df['mean_len']      = list(map(lambda x: x[9], token_features))
que_df['lcs_ratio']     = list(map(lambda x:x[10], token_features))

In [44]:
!pip install fuzzywuzzy

In [45]:
from fuzzywuzzy import fuzz

c:\python\python37\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [46]:
def fetch_fuzzy_features(row):
  q1 = row['question1']
  q2 = row['question2']
  
  fuzzy_features = [0.0]*4
  
  # fuzz_ratio
  fuzzy_features[0] = fuzz.QRatio(q1, q2)

  # fuzz_partial_ratio
  fuzzy_features[1] = fuzz.partial_ratio(q1, q2)

  # token_sort_ratio
  fuzzy_features[2] = fuzz.token_sort_ratio(q1, q2)

  # token_set_ratio
  fuzzy_features[3] = fuzz.token_set_ratio(q1, q2)

  return fuzzy_features

In [47]:
fuzzy_features = que_df.apply(fetch_fuzzy_features, axis=1)


In [48]:
que_df['fuzz_ratio'] = list(map(lambda x: x[0], fuzzy_features))
que_df['fuzz_partial_ratio'] = list(map(lambda x: x[1], fuzzy_features))
que_df['token_sort_ratio'] = list(map(lambda x: x[2], fuzzy_features))
que_df['token_set_ratio'] = list(map(lambda x: x[3], fuzzy_features))

In [49]:
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(que_df['question1']) + list(que_df['question2'])

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [50]:
temp_df1 = pd.DataFrame(q1_arr, index= que_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= que_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(30000, 6000)

In [51]:
final_df = que_df.drop(columns=['question1','question2'])

In [52]:
final_df = pd.concat([final_df, temp_df], axis=1)

In [53]:
final_df.shape

(30000, 6022)

In [54]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(final_df, y ,test_size=0.2)

In [55]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

c:\python\python37\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
c:\python\python37\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


0.7925

In [56]:
def test_common_words(q1, q2):
  w1 = set(map(lambda words : words.lower().strip(), q1.split(" ")))
  w2 = set(map(lambda words : words.lower().strip(), q2.split(" ")))
  return len(w1 & w2)

In [57]:
def test_total_words(q1, q2):
  w1 = set(map(lambda words : words.strip(), q1.split(" ")))
  w2 = set(map(lambda words : words.strip(), q2.split(" ")))
  return (len(w1) + len(w2))

In [58]:
def fetch_test_features(q1, q2):
  SAFE_DIV = 0.00001
  STOP_WORDS = stopwords.words('english')
  token_features = [0.0] * 11

  q1_tokens = q1.split()
  q2_tokens = q2.split()

  if len(q1_tokens) == 0 or len(q2_tokens) == 0:
    return token_features

  q1_words = set([word for word in q1_tokens if word not in STOP_WORDS])
  q2_words = set([word for word in q2_tokens if word not in STOP_WORDS])

  q1_stop = set([word for word in q1_tokens if word in STOP_WORDS])
  q2_stop = set([word for word in q2_tokens if word in STOP_WORDS])

  common_word_cnt = len(q1_words.intersection(q2_words))
  common_stop_cnt = len(q1_stop.intersection(q2_stop))
  common_token_cnt = len(set(q1_tokens).intersection(set(q2_tokens)))

  token_features[0] = round(common_word_cnt / (min(len(q1_words), len(q2_words)) + SAFE_DIV), 2)
  token_features[1] = round(common_word_cnt / (max(len(q1_words), len(q2_words)) + SAFE_DIV), 2)

  token_features[2] = round(common_stop_cnt / (min(len(q1_stop), len(q2_stop)) + SAFE_DIV), 2)
  token_features[3] = round(common_stop_cnt / (max(len(q1_stop), len(q2_stop)) + SAFE_DIV), 2)

  token_features[4] = round(common_token_cnt / (min(len(q1_tokens), len(q2_tokens)) + SAFE_DIV), 2)
  token_features[5] = round(common_token_cnt / (max(len(q1_tokens), len(q2_tokens)) + SAFE_DIV), 2)

  token_features[6] = int(q1_tokens[-1] == q2_tokens[-1])
  token_features[7] = int(q1_tokens[0] == q2_tokens[0])

  token_features[8] = abs(len(q1_tokens) - len(q2_tokens))
  token_features[9] = round((len(q1_tokens) - len(q2_tokens)) / 2, 2)

  strs = list(distance.lcsubstrings(q1, q2))
  token_features[10] = round(len(strs[0]) / (min(len(q1), len(q2)) + 1), 2)
  return token_features

In [59]:
def fetch_test_fuzzy_features(q1, q2):  
  fuzzy_features = [0.0]*4
  
  # fuzz_ratio
  fuzzy_features[0] = fuzz.QRatio(q1, q2)

  # fuzz_partial_ratio
  fuzzy_features[1] = fuzz.partial_ratio(q1, q2)

  # token_sort_ratio
  fuzzy_features[2] = fuzz.token_sort_ratio(q1, q2)

  # token_set_ratio
  fuzzy_features[3] = fuzz.token_set_ratio(q1, q2)

  return fuzzy_features

In [60]:
def query_point_creator(q1, q2):
  input_query = []
  q1 = preprocess(q1)
  q2 = preprocess(q2)

  input_query.append(len(q1))
  input_query.append(len(q2))

  input_query.append(len(q1.split(" ")))
  input_query.append(len(q2.split(" ")))

  input_query.append(test_common_words(q1, q2))
  input_query.append(test_total_words(q1, q2))

  input_query.append(round(input_query[4] / input_query[5], 2))
  
  token_features = fetch_test_features(q1, q2)
  input_query.extend(token_features)

  fuzzy_features = fetch_test_fuzzy_features(q1, q2)
  input_query.extend(fuzzy_features)

  q1_arr = cv.transform([q1]).toarray()
  q2_arr = cv.transform([q2]).toarray()


  return np.hstack((np.array(input_query).reshape(1, 22), q1_arr, q2_arr))

In [64]:
result = query_point_creator("who is the prime minister of India?", "who is the current prime minister of Italy?")
result.shape

(1, 6022)

In [66]:
rf.predict(query_point_creator("who is the prime minister of India?", "who is the current prime minister of India?"))[0]

1

In [67]:
import pickle
pickle.dump(rf, open('model.pkl', 'wb'))
pickle.dump(cv, open('cv.pkl', 'wb'))